# Visualise the results

Run the following codes do visualise the results (only modify the first cell of code)

In [ ]:
import pandas as pd
import ast
import numpy as np

df = pd.read_csv("ADD HERE THE NAME OF THE FILE") 
di = 5 # Add here the specified number of items dropping out of the first 5 recommended items (1-5)
pd.set_option('display.max_rows', None)

def dropped_items(row):
    if pd.isnull(row['topk']) or pd.isnull(row['new_topk']):
        return np.nan
    topk = ast.literal_eval(row['topk'])
    new_topk = ast.literal_eval(row['new_topk'])
    return len(set(topk[:5]) - set(new_topk[:5]))

df = pd.read_csv("ADD HERE THE NAME OF THE FILE")

df['dropped_items'] = df.apply(dropped_items, axis=1)
di = di - 1
df_updated = df[df['dropped_items'] > di]

print("Updated DataFrame:\n", df_updated)

In [ ]:
def count_counterfactuals(row):
    if pd.isnull(row['counterfactual']):
        return np.nan
    counterfactual = ast.literal_eval(row['counterfactual'])
    return len(counterfactual)

df_updated['counterfactual_count'] = df_updated.apply(count_counterfactuals, axis=1)
user_counterfactuals = df_updated.groupby('user')['counterfactual_count'].sum()
print(user_counterfactuals)

In [ ]:
df_updated['counterfactual'] = df_updated['counterfactual'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else set())

total_counterfactuals = df_updated['counterfactual'].apply(len).sum()

num_users_with_counterfactuals = df_updated['counterfactual'].apply(lambda x: len(x) > 0).sum()

result = total_counterfactuals / num_users_with_counterfactuals if num_users_with_counterfactuals != 0 else 0
print("total_counterfactuals:", total_counterfactuals)
print("num_users_with_counterfactuals:", num_users_with_counterfactuals)
print(result)

In [ ]:
import re
import pandas as pd
import numpy as np

# Load the csv files into pandas dataframes
df_topk = df_updated
df_movies = pd.read_csv("movie.csv")

# Create a dictionary mapping movieId to movie title
id_to_title = dict(zip(df_movies['movieId'], df_movies['title']))

def process_counterfactual(x):
    if isinstance(x, str):
        return {id_to_title.get(int(id.strip()), id) for id in x.strip('{}').split(',') if id.strip()}
    else:
        return np.nan

def process_list(x):
    if isinstance(x, str):
        return [id_to_title.get(int(id), int(id)) for id in re.findall(r'\d+', x)][:5]  # Only return first 5 items
    else:
        return np.nan

# Replace movieId with movie title in topk and counterfactual lists
df_topk['counterfactual'] = df_topk['counterfactual'].apply(process_counterfactual)
df_topk['topk'] = df_topk['topk'].apply(process_list)
df_topk['new_topk'] = df_topk['new_topk'].apply(process_list)

# Convert the topk, new_topk, and counterfactual columns back to string
df_topk['topk'] = df_topk['topk'].astype(str)
df_topk['new_topk'] = df_topk['new_topk'].astype(str)
df_topk['counterfactual'] = df_topk['counterfactual'].astype(str)

# Remove rows that only have empty sets in the counterfactual column
df_topk = df_topk[df_topk['counterfactual'] != "nan"]
df_topk

In [ ]:
for index, row in df_topk.iterrows():
    print("\033[1m" + "User" + "\033[0m", row['user'], "\033[1m" + "was recommended:" + "\033[0m", row['topk'], "\n\n", "\033[1m" + "If the following movies are removed (counterfactuals)...:" + "\033[0m",  row['counterfactual'], "\n\n", "\033[1m" + "...the recommendation would be:" + "\033[0m", row['new_topk'])
    print("\n\n")

### Check duplicate scores for each user

In [2]:
import pandas as pd

# Load the data
df = pd.read_csv('recs.csv')

# Group by 'user' and 'score', then count the occurrences
grouped_df = df.groupby(['user', 'score']).size().reset_index(name='occurrences')

# Filter the rows where 'occurrences' is more than 1, these are the duplicate scores
duplicates = grouped_df[grouped_df['occurrences'] > 1]

# Check if there are any duplicates
if duplicates.empty:
    print("No user has matching scores.")
else:
    print("The following users have matching scores:")
    print(duplicates)


The following users have matching scores:
       user     score  occurrences
35144   351  4.091125            2
42406   424  4.327451            2
